In [18]:
import numpy as np
import os.path

In [40]:
class patchFamily2D:
    def __init__(self,file_ID):
        #
        self.U = [] # Knot Vector U
        #self.V = [] # Knot Vector V
        self.CP = [] # Kontrol Points
        self.p = [] # Orders
        self.N = [] # Number of Basis Functions
        # Connectivity
        self.nel = []
        self.nnp = []
        self.nen = []
        self.INC = []
        self.IEN = []
        # Read Patch Data
        file_exists = os.path.exists(file_ID+'1')
        fNo = 1
        while (file_exists == True):
            f = open(file_ID+str(fNo), "r")
            data = f.readlines()
            # Knot Vector - V
            lenV = int(data[0])
            V_p = np.zeros(lenV)
            dataV = data[1].split(" ")
            for i in range(0,lenV):
                V_p[i] = float(dataV[i])
            #self.V.append(V_p)
            # Knot Vector - U
            lenU = int(data[2])
            U_p = np.zeros(lenU)
            dataU = data[3].split(" ")
            for i in range(0,lenU):
                U_p[i] = float(dataU[i])
            self.U.append([])
            self.U[fNo-1].append(U_p)
            self.U[fNo-1].append(V_p)
            # Control Points:
            lenCP = int(data[4])
            CP_p = np.zeros((lenCP,4))
            for i in range(0,lenCP):
                dataCP = data[i+5].split(" ")
                for j in range(0,4):
                    CP_p[i][j] = float(dataCP[j])
            self.CP.append(CP_p)
            # Order:
            p1 = 0
            p2 = 0
            while True:
                if (U_p[p1] == U_p[p1+1]):
                    p1 += 1
                else:
                    break
            while True:
                if (V_p[p2] == V_p[p2+1]):
                    p2 += 1
                else:
                    break  
            self.p.append(np.array([p1,p2]))
            n = len(U_p)-p1-1
            m = len(V_p)-p2-1
            self.N.append(np.array([n,m]))
            # Connectivity:
            nel = (n-p1)*(m-p2)
            self.nel.append(nel)
            nnp = n*m
            self.nnp.append(nnp)
            nen = (p1+1)*(p2+1)
            self.nen.append(nen)
            #
            INC = np.zeros((n*m,2))
            IEN = np.zeros((nel,nen))
            A = 0
            B = 0
            for j in range(1,m+1):
                for i in range(1,n+1):
                    INC[A][0] = i-1
                    INC[A][1] = j-1
                    if (i >= p1+1 and j >= p2+1):
                        for k in range(0,p2+1):
                            for l in range(0,p1+1):
                                C = int(k*(p1+1)+l)
                                IEN[B][C] = int((A-k*n-l))
                        B += 1
                    A += 1
            self.INC.append(INC)
            self.IEN.append(IEN)
            #
            fNo = fNo + 1
            file_exists = os.path.exists(file_ID+str(fNo))
        # Number of Patches:
        self.np = fNo-1
        
    def findSpan(self,pN,pD,u):
        if (u == self.U[pN][pD][self.N[pN][pD]+1]):
            return n
        low = self.p[pN][pD]
        high = self.N[pN][pD]+1
        mid = int((low+high)/2)
        while (u<self.U[pN][pD][mid] or u>self.U[pN][pD][mid+1]):
            if (u<self.U[pN][pD][mid]):
                high = mid
            else:
                low = mid
            mid = int(np.floor((low+high)/2))
        return mid
    
    def dersBasisFuns(self,pN,pD,u,n):
        i = self.findSpan(pN,pD,u)
        left = np.zeros(self.p[pN][pD]+1)
        right = np.zeros(self.p[pN][pD]+1)
        ndu = np.zeros((self.p[pN][pD]+1,self.p[pN][pD]+1))
        a = np.zeros((2,self.p[pN][pD]+1))
        ders = np.zeros((n+1,self.p[pN][pD]+1))
        #
        ndu[0][0] = 1.0
        for j in range(1,self.p[pN][pD]+1):
            left[j] = u-self.U[pN][pD][i+1-j]
            right[j] = self.U[pN][pD][i+j]-u
            saved = 0.0
            for r in range(0,j):
                ndu[j][r] = right[r+1]+left[j-r]
                temp = ndu[r][j-1]/ndu[j][r]
                ndu[r][j] = saved+right[r+1]*temp
                saved = left[j-r]*temp
            ndu[j][j] = saved
        for j in range(0,self.p[pN][pD]+1):
            ders[0][j] = ndu[j][self.p[pN][pD]]
        for r in range(0,self.p[pN][pD]+1):
            s1 = 0
            s2 = 1
            a[0][0] = 1.0
            for k in range(1,n+1):
                d =0.0
                rk = r-k
                pk = self.p[pN][pD]-k
                if (r >= k):
                    a[s2][0] = a[s1][0]/ndu[pk+1][rk]
                    d = a[s2][0]*ndu[rk][pk]
                if (rk >= -1):
                    j1 = 1
                else:
                    j1 = -rk
                if (r-1 <= pk):
                    j2 = k-1
                else:
                    j2 = self.p[pN][pD]-r
                for j in range(j1,j2+1):
                    a[s2][j] = (a[s1][j]-a[s1][j-1])/ndu[pk+1][rk+j]
                    d += a[s2][j]*ndu[rk+j][pk]
                if (r <= pk):
                    a[s2][k] = -a[s1][k-1]/ndu[pk+1][r]
                    d += a[s2][k]*ndu[r][pk]
                ders[k][r] = d
                j = s1
                s1 = s2
                s2 = j
        r = self.p[pN][pD]
        for k in range(1,n+1):
            for j in range(0,self.p[pN][pD]+1):
                ders[k][j] *= r
            r *= (self.p[pN][pD]-k)
        return ders
    
    def ratBasisFuns2D(self,pN,pD,u,v,du,dv):
        N = self.dersBasisFuns(self,pN,pD,u,du)
        M = self.dersBasisFuns(self,pN,pD,v,dv)
        R = np.zeros((((self.p[pN][0],self.p[pN][1],du+1,dv+1))))
        wders = np.zeros((du+1,dv+1))
        for k in range(0,du+1):
            for l in range(0,dv+1):
                wders[k][l] = N[k][:].dot(wMat.dot(M[l][:]))
                temp1 = (N[k][:].dot(M[l][:]))*wMat
                for j in range(1,l+1):
                    temp1 -= (math.comb(l,j)*wders[0][j])*R[:][:][k][l-j]
                for i in range(1,k+1):
                    temp1 -= (math.comb(k,i)*wders[i][0])*R[:][:][k-i][l]
                    temp2 = np.zeros((p,q))
                    for j in range (1,l+1):
                        temp2 += (math.comb(l,j)*wders[i][j])*R[:][:][k-i][l-j]
                    temp1 -= math.comb(k,i)*temp2
                R[:][:][k][l] = temp1/wders[0][0]
        return R

In [41]:
pf = patchFamily2D('igabem_DR05_4th_768_')

In [46]:
pf.dersBasisFuns(0,0,0.3,5)

array([[ 1.70666639e-02,  3.52066662e-01,  5.44733345e-01,
         8.60666620e-02,  6.66665760e-05],
       [-2.04799988e+00, -1.30880002e+01,  8.35200035e+00,
         6.75199975e+00,  3.19999693e-02],
       [ 1.84320007e+02,  1.15199956e+01, -5.64480009e+02,
         3.57120013e+02,  1.15199935e+01],
       [-1.10592015e+04,  3.04128039e+04, -2.48832045e+04,
         2.76480254e+03,  2.76479956e+03],
       [ 3.31776080e+05, -1.32710426e+06,  1.99065636e+06,
        -1.32710426e+06,  3.31776080e+05],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00]])

In [ ]:
pf.V[0][:]

In [8]:
U = []

In [9]:
U.append([])

In [11]:
U.append([])

In [12]:
U

[[], []]